### Count predict time

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import scipy.io
import time

In [ ]:
model_filename = './drive/MyDrive/Thesis/nn_trained/loc_cnn_2.h5'
model = load_model(model_filename,  compile=False)
subject='A0206'
ms='25'
real_data_file = './thesis/real_data/{}/{}ms/eeg_topo_real_{}ms.mat'.format(subject,ms,ms)
topos_eval =  scipy.io.loadmat(real_data_file)['Zi'] 

In [ ]:
topos_eval= np.expand_dims(topos_eval, axis=0)
start_time = time.time()
predicted_sources = model.predict(topos_eval).T
print("--- %s seconds ---" % (time.time() - start_time))

### Load data

In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# to load large .mat files
import mat73
import random

path = './drive/MyDrive/Thesis/single_dipole_locations_cnn/'

In [ ]:
eeg = np.load(path+'eeg.npy')
sources = None
locations = np.load(path+'locations.npy')
fwd = solve_forward(downsampled_dipoles=False,num_dipoles='50k')
sim = Simulation(fwd=fwd, eeg_data=eeg,source_data=sources,locations=locations)

In [ ]:
eeg_topographies = mat73.loadmat(path+'eeg_topos.mat')['eeg_topos']
eeg_topographies = eeg_topographies.transpose(2, 0, 1)

### Experiment

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import (Dense, Dropout, Conv2D,Conv3D, Flatten, MaxPooling2D, Conv1D, BatchNormalization) 

In [ ]:
num_of_simultaneously_dipoles = 1
model = keras.Sequential()

# add input layer
model.add(keras.Input(shape=(eeg_topographies.shape[1], eeg_topographies.shape[2],1), name='Input'))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())            
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5096, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
# Add output layer with only 3 output neurons, one for each coordinate in the 3D-space.
model.add(Dense(3*num_of_simultaneously_dipoles, activation='relu'))

model.summary()


In [ ]:
'''
 ' Huber loss.
 ' https://en.wikipedia.org/wiki/Huber_loss
'''
def huber_loss(y_true, y_pred, clip_delta=1.0):
  error = y_true - y_pred
  cond  = tf.keras.backend.abs(error) < clip_delta

  squared_loss = 0.5 * tf.keras.backend.square(error)
  linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)

  return tf.where(cond, squared_loss, linear_loss)

'''
 ' Same as above but returns the mean loss.
'''
def huber_loss_mean(y_true, y_pred, clip_delta=1.0):
  return tf.keras.backend.mean(huber_loss(y_true, y_pred, clip_delta))

In [ ]:
loss = tf.keras.losses.MeanAbsoluteError()

metrics = [#tf.keras.metrics.MeanAbsolutePercentageError(name="MAPE"),
            #tf.keras.losses.MeanAbsoluteError(name='MAE')
            'mse'         
        ]

#lr_callback = keras.callbacks.LearningRateScheduler(NN.lr_schedule)
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

model.compile(optimizer, loss, metrics=metrics)

 # Input data
x = eeg_topographies       

# Target data (3D locations in the source space)
y = sim.locations        

# early stoping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', \
    mode='min', patience=250, restore_best_weights=True,verbose=1)

history = model.fit(x, y, 
                epochs=500, batch_size=32, shuffle=True, 
                validation_split=0.2, callbacks=[es])

In [ ]:
model.save('./drive/MyDrive/Thesis/nn_trained/loc_cnn_2.h5')

In [ ]:
import pandas as pd
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

hist_csv_file = './drive/MyDrive/Thesis/nn_trained/history_loc_cnn_2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

### LocCNN

In [ ]:
from net import LocCNN

eeg_cnn = LocCNN(sim=sim,eeg_topographies=eeg_topographies,verbose=False)
eeg_cnn.build_model()

In [ ]:
history = eeg_cnn.fit(batch_size=16)

In [ ]:
eeg_cnn.save_nn('./drive/MyDrive/Thesis/nn_trained/loc_cnn_1.h5')